# Using LLMs with Structured Data

In [ ]:
from langchain.agents import create_sql_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
llm = OpenAI()
db = SQLDatabase.from_uri('sqlite:///chinook.db')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

[tool.name for tool in toolkit.get_tools()]

#### ['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

- sql_db_query: This takes as input a detailed and correct SQL query, and it outputs a result from the database. If the query is not correct, an error message will be returned.
- sql_db_schema: This takes as input a comma-separated list of tables, and it outputs the schema and sample rows for those tables.
- sql_db_list_tables: This takes as input an empty string, and it outputs a comma-separated list of tables in the database.
- sql_db_query_checker: This tool double-checks whether the query is correct before executing it.

In [ ]:
agent_executor.run("Describe the playlisttrack table")

#### Entering new AgentExecutor chain...
- Action: sql_db_list_tables
- Action Input:
- Observation: album, artist, customer, employee, genre, invoice, invoice_line, media_type, playlist, playlist_track, track
- Thought: The table I need is playlist_track
- Action: sql_db_schema
- Action Input: playlist_track
- Observation:
- CREATE TABLE playlist_track (
- [...]
- > Finished chain.
#### 'The playlist_track table contains the playlist_id and track_id columns. It has a primary key of playlist_id and track_id. There is also a foreign key reference to the track and playlist tables. Sample rows include (1, 3402), (1, 3389), and (1, 3390).'

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

#### Thanks to this prompt template, the agent is able to use the proper tools and generate a SQL query, without modifying the underlying database (you can see the explicit rule not to run any data manipulation language (DML) statements).

#### DML is a class of SQL statements that are used to query, edit, add, and delete row-level data from database tables or views. The main DML statements are as follows:

- SELECT: This is used to retrieve data from one or more tables or views based on specified criteria.
- INSERT: This is used to insert new data records or rows into a table.
- UPDATE: This is used to modify the values of existing data records or rows in a table.
- DELETE: This is used to remove one or more data records or rows from a table.
- MERGE: This is used to combine the data from two tables into one based on a common column.
- DML statements are used to store, modify, retrieve, delete, and update data in a database.

## Prompt engineering

#### As we saw in the previous chapter, pre-built LangChain agents and chains come with default prompts, which make it easier to tailor them toward their goals. Nevertheless, we can customize that prompt and pass it as a parameter to our component. For example, let’s say that we want our SQL agent to print the SQL query it used to return the result.